In [1]:
from discussion_agents.cog.functional.self_refine import _prompt_agent, _prompt_feedback, _prompt_refine
from discussion_agents.cog.prompts.self_refine import (
    GSM8K_FEWSHOT_EXAMPLES, 
    GSM8K_FEEDBACK_FEWSHOT_EXAMPLES,
    GSM8K_REFINE_FEWSHOT_EXAMPLES,
    SELF_REFINE_INSTRUCTION_GSM8K,
    SELF_REFINE_FEEDBACK_INSTRUCTION_GSM8K,
    SELF_REFINE_REFINE_INSTRUCTION_GSM8K,
    SELF_REFINE_FEEDBACK_EXAMPLE_FORMAT_GSM8K
)
from discussion_agents.cog.agent.self_refine import SelfRefineAgent
from langchain_community.chat_models.openai import ChatOpenAI

import dotenv
import os

dotenv.load_dotenv("../.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    openai_api_key=openai_api_key
)
question = "A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?"

In [ ]:
agent = SelfRefineAgent(llm=llm)

In [ ]:
out = agent.generate(
    question=question
)